In [33]:
from geopy.geocoders import Nominatim
import pandas as pd
import time
import requests


In [34]:
df = pd.read_csv('data.csv')

In [35]:
import re

def preprocess_address(address):
    
    address = re.sub(r'\(.*?\)', '', address)
    
    address = address.replace("д.", "дом").replace("ш.", "шоссе").replace("ул.", "улица")
    
    address = re.sub(r'\s+', ' ', address).strip().strip(',')
    return address

df['Адрес'] = df['Адрес'].apply(preprocess_address)

In [36]:
df['Адрес'].unique()

array(['Москва, дом Бачурино, 254, шоссе Калужское , шоссе Варшавское',
       'Москва, Дмитровское шоссе, 122Гк10, шоссе Дмитровское , шоссе Москва-Санкт-Петербург, M-11',
       'Москва, Краснопахорское поселение, дом Романцево, Романтика кп, шоссе Калужское , шоссе Варшавское',
       'Москва, Сосенское поселение, Филатов Луг-2 СНТ, шоссе Киевское , шоссе Калужское',
       'Москва, Десеновское поселение, шоссе Калужское , шоссе Варшавское',
       'Москва, Крылатская улица, 40',
       'Москва, Первомайское поселение, дом Пучково, улица Новая, 4, шоссе Калужское , шоссе Киевское',
       'Москва, Первомайское поселение, шоссе Калужское , шоссе Киевское',
       'Москва, Вороновское поселение, пос. ЛМС, улица Луговая, 19, шоссе Калужское , шоссе Варшавское',
       'Москва, Краснопахорское поселение, дом Романцево, улица Лесная, 21, шоссе Калужское , шоссе Киевское',
       'Москва, 2-я Лыковская улица',
       'Москва, Рязановское поселение, № 15 кв-л, шоссе Симферопольское , шоссе

In [37]:
api_key = "your_api_key"

def get_coordinates(address, api_key):
    """
    Получить широту и долготу для адреса с использованием Яндекс Геокодера.
    """
    url = f"https://geocode-maps.yandex.ru/1.x/"
    params = {
        "apikey": api_key,
        "geocode": address,
        "format": "json",
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        result = response.json()
       
        pos = result["response"]["GeoObjectCollection"]["featureMember"][0]["GeoObject"]["Point"]["pos"]
        lon, lat = pos.split(" ")
        return float(lat), float(lon)
    except (IndexError, KeyError):
        return None, None  
    except requests.RequestException as e:
        print(f"Ошибка при запросе: {e}")
        return None, None

In [38]:
df["Широта"], df["Долгота"] = zip(*df["Адрес"].apply(lambda x: get_coordinates(x, api_key)))


In [39]:
df

,"Площадь, м2",Количество спален,Участок,Адрес,Телефоны,Описание,Цена,Можно с детьми/животными,Дополнительно,Ссылка на объявление,Широта,Долгота
0,380.0,4.0,"12.0, сот.","Москва, дом Бачурино, 254, шоссе Калужское , ш...",+79853345356,"Уютный, светлый и современный двухэтажный дом,...","25000.0 руб./ За день, Залог - 30000 руб.",NaN,"Мебель в комнатах, Мебель на кухне, Баня, Басс...",https://www.cian.ru/rent/suburban/306675289,55.445135,37.289867
1,500.0,9.0,"30.0, сот.","Москва, Дмитровское шоссе, 122Гк10, шоссе Дмит...",+79623641352,"Уличный, открытый бассейн с горячей водой и зи...","6000.0 руб./ За день, Залог - 7000 руб.",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Баня, Басс...",https://www.cian.ru/rent/suburban/279568123,55.922126,37.548411
2,500.0,5.0,"15.0, сот.","Москва, Краснопахорское поселение, дом Романце...",+79154560650,"# 177\n\nСдаем новый коттедж 500 кв.м. в КП "" ...","25000.0 руб./ За день, Залог - 20000 руб.","Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Баня, Басс...",https://www.cian.ru/rent/suburban/301776830,55.445135,37.289867
3,560.0,7.0,"20.0, сот.","Москва, Сосенское поселение, Филатов Луг-2 СНТ...",+79154560650,КОТТЕДЖ #84 \n\nСдается в аренду посуточно 4-...,"25000.0 руб./ За день, Залог - 20000 руб.","Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Баня, Басс...",https://www.cian.ru/rent/suburban/229484352,55.445135,37.289867
4,750.0,11.0,"30.0, сот.","Москва, Десеновское поселение, шоссе Калужское...",+79855840197,#23505\n3 ОСНОВНЫХ ПРЕИМУЩЕСТВА:\n-вместительн...,"29000.0 руб./ За день, Залог - 15000 руб.","Можно с детьми, Можно с животными","Мебель в комнатах, Мебель на кухне, Баня, Басс...",https://www.cian.ru/rent/suburban/265389810,55.445135,37.289867
...,...,...,...,...,...,...,...,...,...,...,...,...
195,80.0,NaN,"1.0, сот.","Москва, № 55 кв-л, 15, шоссе Калужское , шоссе...",+74952235776,"Грядок.Нет находится в Новой Москве, вблизи де...","14280.0 руб./ За день, Залог - 10000 руб.",Можно с детьми,"Мебель в комнатах, Телевизор, Интернет",https://www.cian.ru/rent/suburban/310118442,55.562678,37.441296
196,80.0,NaN,"1.0, сот.","Москва, № 55 кв-л, 15, шоссе Калужское , шоссе...",+74952235776,"Грядок.Нет находится в Новой Москве, вблизи де...","14280.0 руб./ За день, Залог - 10000 руб.",Можно с детьми,"Мебель в комнатах, Телевизор, Интернет",https://www.cian.ru/rent/suburban/310112729,55.562678,37.441296
197,200.0,5.0,"8.0, сот.","Москва, Щербинка, улица Парковая, 32А, шоссе С...",+79858042828,Сдается дом посуточно от собственника!!! Подхо...,"15000.0 руб./ За день, Залог - 12000 руб.",Можно с детьми,"Мебель на кухне, Баня, Душевая кабина, Стираль...",https://www.cian.ru/rent/suburban/310643953,55.680402,37.622603
198,420.0,7.0,"20.0, сот.","Москва, Первомайское поселение, шоссе Калужско...","+79645905898, +79651102869","СДАЁТСЯ коттедж на сутки, выходные, праздники,...","15000.0 руб./ За день, Залог - 5000 руб.",Можно с детьми,"Мебель в комнатах, Мебель на кухне, Баня, Басс...",https://www.cian.ru/rent/suburban/154210827,55.445135,37.289867


In [40]:
df.to_csv('data.csv', index = False)